In [103]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, datasets, Model, Sequential, optimizers, losses
import numpy as np
from  PIL import Image

### Fashion MNIST 实践练习

In [104]:
batchsz = 128
shuffles = 10000
lr = 1e-3
h_dim = 20
epochs = 100

In [105]:
# 将多张图片合并并保存为一张大图
def save_images(imgs, name):
    
    new_im = Image.new('L', (280, 280))
    index = 0
    for i in range(0, 280, 28):
        for j in range(0, 280, 28):
            im = imgs[index]
            im = Image.fromarray(im, mode='L')
            new_im.paste(im, (i, j))
            index += 1
    new_im.save(name)

In [106]:
#以图片数据为例，适用于全连接层
def preprocess(x):
    """
    自定义预处理函数
    参数：
    x -- 待处理的数据集,维度[b,28,28]
    y -- 待处理的数据集,[b]

    返回值：
    x -- 标准化和扁平化后的 x
    y -- 转换为 one_hot 向量
    """
    x = tf.cast(x, dtype=tf.float32) / 255. #标准化0-1
    #扁平化-不建议，后续需要用 squeeze函数处理。者此处不设，而在输入的时候对x设置x.reshape(x,(-1, 28*28))
    #  注意与 [-1, 28*28]的区别
    x = tf.reshape(x, [28*28])


    return x

自编码器：

<img src="img/01.jpg"/>

- 编码器：3 层全连接层网络，输出节点为 256、128、20，
- 解码器：3 层全连接网络，，输出节点为 128、256、784

In [107]:
(x,y), (x_test, y_test)=datasets.fashion_mnist.load_data()
print('数据集fabshionmnist中x的维度', x.shape, '数据集fashionmnist中y的维度', y.shape)
#数据加载后，需要转换为 Dataset对象
train_db = tf.data.Dataset.from_tensor_slices(x)
test_db = tf.data.Dataset.from_tensor_slices(x_test)

# db = db.step1().step2().step3()
# shuffle(缓冲区大小，一般是一个较大的常数)，btach(一批的样本数量)
train_db = train_db.map(preprocess).shuffle(shuffles).batch(batchsz)
test_db = test_db.map(preprocess).batch(batchsz)
print(train_db, test_db)

数据集fabshionmnist中x的维度 (60000, 28, 28) 数据集fashionmnist中y的维度 (60000,)
<BatchDataset shapes: (None, 784), types: tf.float32> <BatchDataset shapes: (None, 784), types: tf.float32>


In [108]:
# 编码器
class autoCode(Model):
    def __init__(self):
        super(autoCode, self).__init__()
        # 创建 Encoders 网络
        self.encoder = Sequential([
            layers.Dense(256, activation=tf.nn.relu),
            layers.Dense(128, activation=tf.nn.relu),
            layers.Dense(h_dim)
        ])
        # 创建 Decoders 网络
        self.decoder = Sequential([
            layers.Dense(128, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(784, activation='relu')
        ])
    # 在 call 函数中实现前向传播过程
    def call(self, inputs, training=None):
        # 编码过程
        # [b, 28*28]=>[b, 20],获得隐藏向量 h
        h = self.encoder(inputs)
        # 重构图片：[b, 20]=>[b, 784(28*28)]
        x_bar = self.decoder(h)
        return x_bar

网络训练

In [109]:
optimizer = optimizers.Adam(learning_rate=lr)
model = autoCode()
model.build(input_shape=[4, 784])
# 参数量
# Encoder:236 436
# Decoder:237 200
model.summary()

Model: "auto_code_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_22 (Sequential)   (4, 20)                   236436    
_________________________________________________________________
sequential_23 (Sequential)   (4, 784)                  237200    
Total params: 473,636
Trainable params: 473,636
Non-trainable params: 0
_________________________________________________________________


In [111]:
for epoch in range(epochs):
    for step, x in enumerate(train_db):
        with tf.GradientTape() as tape:
            logits = model(x)
            loss = losses.binary_crossentropy(x, logits, from_logits=True)
            loss = tf.reduce_mean(loss)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 10 ==0 and step % 100 == 0:
            print(epoch, step, float(loss))

        # evaluation
        x = next(iter(test_db))
        x_rel = tf.reshape(x, [-1, 28, 28])
        # print(x.shape)
        logits = model(x)
        x_bar = tf.sigmoid(logits)
        # [b, 28*28]=>[b, 28, 28]
        x_bar = tf.reshape(x_bar, [-1, 28, 28])

        # [b, 28, 28]=>[2b, 28, 28]
        x_concat = tf.concat([x_rel[:50], x_bar[:50]], axis=0)
        # x_concat = x_bar
        x_concat = x_concat.numpy() * 255.
        x_concat = x_concat.astype(np.uint8)
        save_images(x_concat, 'autoCode/epoch_%d.png'%epoch)

0 0 0.6552934646606445
0 1 0.6603056788444519
0 2 0.6486604809761047
0 3 0.6527636647224426
0 4 0.653347373008728
0 5 0.6527901887893677
0 6 0.6567032337188721
0 7 0.6521627902984619
0 8 0.6601362228393555
0 9 0.6544395089149475
0 10 0.653978168964386
0 11 0.6511164903640747
0 12 0.6501554250717163
0 13 0.658989429473877
0 14 0.6487768888473511
0 15 0.6569609642028809
0 16 0.6536020040512085
0 17 0.6570005416870117
0 18 0.6524364948272705
0 19 0.658690333366394
0 20 0.6544895172119141
0 21 0.6537220478057861
0 22 0.6541829705238342
0 23 0.6582502126693726
0 24 0.6529052257537842
0 25 0.655946671962738
0 26 0.6517962217330933
0 27 0.6554372906684875
0 28 0.66205233335495
0 29 0.6502474546432495
0 30 0.6547035574913025
0 31 0.6532195806503296
0 32 0.6550509929656982
0 33 0.6494882106781006
0 34 0.6574854254722595
0 35 0.6575288772583008
0 36 0.6563966274261475
0 37 0.6578041911125183
0 38 0.6525934338569641
0 39 0.6514668464660645
0 40 0.6562644839286804
0 41 0.6531639099121094
0 42 0.65

KeyboardInterrupt: 

In [ ]:
model = autoCode()
model.compile(optimizer=optimizer, loss=losses.categorical_crossentropy, metrics=['accuracy'])
history = model.fit(train_db, epochs=epochs, validation_data=test_db)
model.evaluate(test_db)